In [73]:
import pandas as pd
import xarray as xr
import numpy as np

In [82]:
# pdf = pd.read_csv(f'../data/gridcell_studies_6 - Precipitation - upper_pred_2.5.csv').rename(columns={
#     "da_cat": "precip_da_cat",
#     "n_study_prop": "precip_study_prop"
# })
df = pd.read_csv(f'../data/gridcell_studies_6 - Temperature - upper_pred_2.5.csv').rename(columns={
    "da_cat": "temp_da_cat",
    "n_study_prop": "temp_study_prop"
})
# print(tdf.shape)

# df = (pdf[["LAT","LON","precip_da_cat","precip_study_prop"]]
#       .merge(tdf[["LAT","LON","temp_da_cat","temp_study_prop"]])
#      )#.fillna(0)

df.loc[df['LON']>180,"LON"]-=360

df['LAT_5'] = df['LAT']//5*5+5*0.5
df['LON_5'] = df['LON']//5*5+5*0.5



new_temp = (
    xr
    .open_dataset('../data/d_a/tas_attribution_categories_g05_centred.nc', decode_times=False)
)
new_temp = new_temp.to_dataframe().reset_index()

ndf = df.merge(new_temp.rename(columns={"latitude":"LAT_5", "longitude": "LON_5"}), how="left")

ndf.head()

,LAT,LON,n_studies,LAT_25,LON_25,temp_da_cat,index,temp_study_prop,LAT_5,LON_5,Attribution_Category_2016,Attribution_Category_2010
0,-88.75,-178.75,0.0,-87.5,182.5,NaN,0,0.0,-87.5,-177.5,NaN,NaN
1,-88.75,-176.25,0.0,-87.5,182.5,NaN,1,0.0,-87.5,-177.5,NaN,NaN
2,-88.75,-173.75,0.0,-87.5,187.5,NaN,4,0.0,-87.5,-172.5,NaN,NaN
3,-88.75,-171.25,0.0,-87.5,187.5,NaN,5,0.0,-87.5,-172.5,NaN,NaN
4,-88.75,-168.75,0.0,-87.5,192.5,NaN,8,0.0,-87.5,-167.5,NaN,NaN


In [83]:
df = pd.read_csv(f'../data/study_da_6 - Temperature - upper_pred_{degrees}.csv')
print(df.shape)
df.head()

(40116, 234)


,Unnamed: 0,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,random_sample,...,2 - 2.5. Detection of a regional climate trend (no attribution) - std_prediction,2 - 2.5. Detection of a regional climate trend (no attribution) - lower_pred,2 - 2.5. Detection of a regional climate trend (no attribution) - upper_pred,df_da,gridcells,da_trend_cells,da_data_cells,feature_type,new_da_trend_cells,new_da_data_cells
0,0,701728,The cryosphere of mountain regions is highly s...,Mapping the snow line altitude for large glaci...,NaN,NaN,NaN,0.0,0.0,0,...,0.057526,0.122916,0.237967,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,384698,Droughts and landfalling tropical cyclones (TC...,Spatiotemporal patterns of drought/tropical cy...,NaN,NaN,NaN,0.0,0.0,0,...,0.009665,0.055711,0.075041,NaN,1.0,0.0,1.0,OTHER,1.0,1.0
2,9,216497,Sea surface temperatures (SSTs) in all regions...,El-Niño and coral reefs,NaN,NaN,NaN,0.0,0.0,0,...,0.004904,0.035264,0.045071,NaN,1.0,1.0,1.0,OTHER,1.0,1.0
3,11,180236,High-latitude warming has had a discernible ef...,Does summer warming reduce black spruce produc...,NaN,NaN,NaN,0.0,0.0,0,...,0.002401,0.010370,0.015173,NaN,42.0,25.0,34.0,ADM1,30.0,30.0
4,14,461327,Two IPCC fourth assessment report climate mode...,Does the Arctic sea ice have a tipping point?,NaN,"['Geosciences, Multidisciplinary']",NaN,0.0,0.0,0,...,0.007506,0.053730,0.068743,NaN,637.0,10.0,23.0,OCN,17.0,39.0


In [84]:
degrees = 2.5

df_ndf = pd.read_csv(f'../data/study_gridcell_6 - Temperature - upper_pred_{degrees}.csv')

df_ndf.head()

,ndf_id,doc_id
0,7574.0,486888.0
1,8660.0,230251.0
2,8661.0,230251.0
3,8666.0,230251.0
4,8667.0,230251.0


In [91]:
grid_vars = ['gridcells','new_da_trend_cells','new_da_data_cells']
for i, (did, group) in enumerate(df_ndf.groupby('doc_id')):
    cells = ndf[ndf['index'].isin(group['ndf_id'])]  
    trend_cells = cells[(cells['Attribution_Category_2016']>1) & (~pd.isna(cells['Attribution_Category_2016']))].shape[0]
    data_cells = cells[~pd.isna(cells['Attribution_Category_2016'])].shape[0]
    
    
    df.loc[df['id']==did, grid_vars] = [cells.shape[0], trend_cells, data_cells]

In [88]:
cells

,LAT,LON,n_studies,LAT_25,LON_25,temp_da_cat,index,temp_study_prop,LAT_5,LON_5,Attribution_Category_2016,Attribution_Category_2010
5803,11.25,-71.25,5.0,12.5,287.5,2.0,5845,2.420631,12.5,-72.5,2.0,3.0


In [87]:
df.loc[df['id']==did]

,Unnamed: 0,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,random_sample,...,2 - 2.5. Detection of a regional climate trend (no attribution) - std_prediction,2 - 2.5. Detection of a regional climate trend (no attribution) - lower_pred,2 - 2.5. Detection of a regional climate trend (no attribution) - upper_pred,df_da,gridcells,da_trend_cells,da_data_cells,feature_type,new_da_trend_cells,new_da_data_cells
32215,96644,266,"Paleoclimate of La Guajira, Colombia; by the g...","Paleoclimate of La Guajira, Colombia; by the g...",dendroclimatology; annual tree rings in tropic...,['Biology'],NaN,0.0,0.0,0,...,0.008251,0.040661,0.057164,NaN,0.0,1.0,1.0,ADM1,1.0,1.0


In [90]:
df[df['da_trend_cells']>df['gridcells']]

,Unnamed: 0,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,random_sample,...,2 - 2.5. Detection of a regional climate trend (no attribution) - std_prediction,2 - 2.5. Detection of a regional climate trend (no attribution) - lower_pred,2 - 2.5. Detection of a regional climate trend (no attribution) - upper_pred,df_da,gridcells,da_trend_cells,da_data_cells,feature_type,new_da_trend_cells,new_da_data_cells
40,131,1553235,"During 1998-2011, in an area of western Cyprus...",A sustained decline in Cyprus Warbler Sylvia m...,NaN,['Biodiversity Conservation; Ornithology'],NaN,0.0,0.0,0,...,0.010465,0.061281,0.082211,NaN,0.0,1.0,1.0,PCLI,1.0,1.0
163,468,2313456,This work analyses the influence of NAO on the...,Influence of Winter North Atlantic Oscillation...,Snow; North Atlantic Oscillation (NAO); Precip...,['Environmental Sciences'],NaN,0.0,0.0,0,...,0.039768,0.091132,0.170667,NaN,0.0,1.0,1.0,PCLI,1.0,1.0
264,754,1810950,A dendroclimatic reconstruction is carried out...,Dendroclimatic reconstruction of the precipita...,Fagus sylvatica; dendrochronology; dendroclima...,['Geography'],NaN,0.0,0.0,0,...,0.023549,0.071148,0.118246,NaN,0.0,1.0,1.0,ADM1,1.0,1.0
265,756,94477,From paleoecological records it is well known ...,Selection and use of bio-indicators to assess ...,bio-indicators; phenology; distribution; clima...,"['Ecology; Geography, Physical']",NaN,0.0,0.0,0,...,0.010964,0.039113,0.061041,NaN,0.0,1.0,1.0,PCLI,1.0,1.0
273,775,3949608,"Honduras is a country with an area of 68,000 s...",Honduras.,NaN,NaN,NaN,0.0,0.0,0,...,0.003320,0.022330,0.028971,NaN,0.0,1.0,1.0,PCLI,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39989,119883,97447,Here we present the first high-resolution delt...,Cuban stalagmite suggests relationship between...,Atlantic Multidecadal Oscillation; Caribbean; ...,"['Geography, Physical; Geosciences, Multidisci...",NaN,0.0,0.0,0,...,0.039975,0.110958,0.190908,NaN,0.0,1.0,1.0,PCLI,1.0,1.0
40017,119957,771610,Purpose - What people understand by climate ch...,Perception and responses of traders to climate...,Coping; Ghana; Climate change; Livelihoods; Ba...,['Environmental Studies'],NaN,0.0,0.0,0,...,0.005799,0.023996,0.035593,NaN,0.0,1.0,1.0,ADM1,1.0,1.0
40076,120161,2458572,We have tested brain size diversity and enceph...,Brain size diversity in adaptation and speciat...,NaN,NaN,NaN,0.0,0.0,0,...,0.006633,0.031356,0.044622,NaN,0.0,1.0,1.0,PCLI,1.0,1.0
40092,120204,3572967,"Prosopis ferox grows in upper-elevation, inter...","Climate, land-use and Prosopis ferox recruitme...",NaN,NaN,NaN,0.0,0.0,0,...,0.008148,0.058123,0.074420,NaN,0.0,1.0,1.0,ADM1,0.0,0.0


In [92]:
df.to_csv(f'../data/study_da_6 - Temperature - upper_pred_{degrees}.csv', index=False)
df.head()

,Unnamed: 0,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,random_sample,...,2 - 2.5. Detection of a regional climate trend (no attribution) - std_prediction,2 - 2.5. Detection of a regional climate trend (no attribution) - lower_pred,2 - 2.5. Detection of a regional climate trend (no attribution) - upper_pred,df_da,gridcells,da_trend_cells,da_data_cells,feature_type,new_da_trend_cells,new_da_data_cells
0,0,701728,The cryosphere of mountain regions is highly s...,Mapping the snow line altitude for large glaci...,NaN,NaN,NaN,0.0,0.0,0,...,0.057526,0.122916,0.237967,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,384698,Droughts and landfalling tropical cyclones (TC...,Spatiotemporal patterns of drought/tropical cy...,NaN,NaN,NaN,0.0,0.0,0,...,0.009665,0.055711,0.075041,NaN,1.0,0.0,1.0,OTHER,1.0,1.0
2,9,216497,Sea surface temperatures (SSTs) in all regions...,El-Niño and coral reefs,NaN,NaN,NaN,0.0,0.0,0,...,0.004904,0.035264,0.045071,NaN,1.0,1.0,1.0,OTHER,1.0,1.0
3,11,180236,High-latitude warming has had a discernible ef...,Does summer warming reduce black spruce produc...,NaN,NaN,NaN,0.0,0.0,0,...,0.002401,0.010370,0.015173,NaN,42.0,25.0,34.0,ADM1,30.0,30.0
4,14,461327,Two IPCC fourth assessment report climate mode...,Does the Arctic sea ice have a tipping point?,NaN,"['Geosciences, Multidisciplinary']",NaN,0.0,0.0,0,...,0.007506,0.053730,0.068743,NaN,637.0,10.0,23.0,OCN,17.0,39.0
